# Scraping article content

In [1]:
import numpy as np
import pandas as pd
import requests
import re
import time
import openai
import redis
import json
import sqlite3
import scrapy
from scrapy.crawler import CrawlerProcess
import bs4

In [2]:
class NewsamListCrawler(scrapy.Spider):
    name = "MovsisyansNewsCrawler"

    custom_settings=dict(
        DOWNLOAD_DELAY = 0.08
    )
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.domain = r"https://news.am/eng/news/"
        self.r = redis.Redis("localhost", 5211, decode_responses=True)

    def start_requests(self):
        while (to_fetch := self.r.rpop("to_fetch_newsam")) is not None:
            yield scrapy.Request(url=self.domain + to_fetch + ".html", callback=self.parse_article)

    def parse_article(self, response):
        resp_text = " ".join(response.text.split())
        
        # print(f"SCRAPING {response.url}")
        soup = bs4.BeautifulSoup(resp_text)

        content = " ".join(soup.find("div", {"class": "article-text"}).findAll(string=True)).strip()
        head = soup.find("div", {"class": "info-short"})
        try:
            region, theme = " ".join(head.findAll(string=True)).split("Theme:")
        except:
            region, theme = "", ""
        else:
            try:
                region = region.split("Region:")[1].strip()
            except:
                region = ""
            theme = theme.strip()

        self.r.lpush("to_store_newsam", json.dumps({"id": response.url.split("/")[-1].split(".")[0], "content": content, "region": region, "theme": theme}))
        

In [3]:
proc = CrawlerProcess()
proc.crawl(NewsamListCrawler)
proc.start()

2023-11-24 10:17:03 [scrapy.utils.log] INFO: Scrapy 2.11.0 started (bot: scrapybot)
2023-11-24 10:17:03 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.11.3 (tags/v3.11.3:f3909b8, Apr  4 2023, 23:49:59) [MSC v.1934 64 bit (AMD64)], pyOpenSSL 23.3.0 (OpenSSL 3.1.4 24 Oct 2023), cryptography 41.0.5, Platform Windows-10-10.0.22621-SP0
2023-11-24 10:17:03 [scrapy.addons] INFO: Enabled addons:
[]
2023-11-24 10:17:03 [py.warnings] WARNING: c:\Projects\NewsEntityTracking\.venv\Lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documen

In [ ]:
response = requests.get("https://news.am/eng/news/169.html")
resp_text = " ".join(response.text.split())
soup = bs4.BeautifulSoup(resp_text)


content = " ".join(soup.find("div", {"class": "article-text"}).findAll(string=True)).strip()
head = soup.find("div", {"class": "info-short"})

region, theme = " ".join(head.findAll(string=True)).split("Theme:")
region = region.split("Region:")[1].strip()
theme = theme.strip()

region, theme, content

In [ ]:
len(" ".join(head.findAll(string=True)).split("Theme:"))

In [ ]:
response.css('span[itemprop="articleBody"].last-child')